In [1]:
from llama_cpp import Llama

In [2]:
%%time
MODEL_PATH_MISTRAL = "models/mistral-7b-instruct-v0.2.Q3_K_M.gguf"
default_model = Llama(
    model_path=MODEL_PATH_MISTRAL,
    device="cuda",
    n_gpu_layers=-1,
    n_ctx=2048,
    verbose=False,
)

CPU times: user 59.8 ms, sys: 1.05 s, total: 1.11 s
Wall time: 25.7 s


In [9]:
annotations_to_use = [
    "Alert",
    "Excited",
    "Happy",
    "Content",
    "Relaxed",
    "Calm",
    "Bored",
    "Depressed",
    "Sad",
    "Distressed",
    "Angry",
    "Tense"
]

prompt_template_default = """
[INST]

<<SYS>>
You are a helpful assistant. 
Always answer the user's questions in brief and clear sentences. 
Keep your outputs short while being helpful.
<</SYS>>

%%%prompt%%%

[/INST]
"""

def query_llm(prompt, enforce_json=True, model=default_model):
    if enforce_json is True:
        response = model.create_chat_completion(
            messages=[
                {
                    "role": "system",
                    "content": f"""
                You are a sentiment analysis model. 
                You are asked to analyze the sentiment of a song based on its lyrics.
                You have to predict the following emotions with a boolean value: 
                {", ".join(annotations_to_use)}
                """,
                },
                {"role": "user", "content": prompt},
            ],
            response_format={
                "type": "json_object",
                "schema": {
                    "type": "object",
                    "properties": dict.fromkeys(annotations_to_use, {"type": "boolean"}),
                    "required": annotations_to_use,
                },
            },
            temperature=0.7,
        )
        return response["choices"][0]["message"]["content"]
    else:
        response = model(
            prompt_template_default.replace("%%%prompt%%%", prompt),
            max_tokens=20000,
            temperature=0.7,
        )
        return response["choices"][0]["text"]

In [10]:
%%time
query_llm("Who is the Prime Minister of India? Answer in 1 sentence", False, default_model)

CPU times: user 18.2 s, sys: 495 ms, total: 18.7 s
Wall time: 3.26 s


'Narendra Modi is the current Prime Minister of India as of 2023.'

In [24]:
def test_lyrics(input_lyrics):
    output = query_llm(input_lyrics)
    output = eval(test_output_1.replace("true", "True").replace("false", "False"))
    # Print all keys in output where value is True
    for k, v in output.items():
        print(k)

In [25]:
test_lyric_1 = """
Then I woke up in love (woke up in love)
Then I woke up in love
Lately, I've been losin' these nights
One more heartbreak and I'm done with this
Shadows on the ceilin', they don't fight
But I still argue for the hell of it
I'm tryin' my hardest now, I feel you reachin' out
I don't wanna be the one that's always on the edge
You found me all messed up
You found me down on my luck
So lost, but then I woke up in love
When I was chasin' highs
You went and saved my life
You picked me up, thank God I woke up in love
With you
Thought I was lost, but then I woke up in love
With you
Thought I was lost, but then I woke up in love (eh)
Then I woke up in love (eh-eh)
Then I woke up in love (eh)
Then I woke up in love (eh-eh)
Then I woke up in love

Sometimes all those dark days, well, they try to find me
But that's when I lean on you
Whatever happens, my trouble's behind me, 'cause
Your love's gonna see me through
Nothing can bring me down, I feel you reachin' out
You're pullin' me back when I'm about to reach the edge
You found me all messed up
You found me down on my luck
So lost, but then I woke up in love
When I was chasin' highs
You went and saved my life
You picked me up, thank God I woke up in love
With you
Thought I was lost, but then I woke up in love
With you
Thought I was lost, but then I woke up in love (eh)
Then I woke up in love (eh-eh)
Then I woke up in love (eh)
Then I woke up in love (eh-eh)
Then I woke up in love
You found me all messed up (all messed up)
You found me down on my luck
So lost, but then I woke up in love (then I woke up in love)
When I was chasin' highs

You went and saved my life
You picked me up, thank God I woke up in love (then I woke up in love)
With you
Thought I was lost, but then I woke up in love (woke up in love)
With you
Thought I was lost, but then I woke up in love
"""

# test_output_1 = query_llm(test_lyric_1)
# test_output_1 = eval(test_output_1.replace("true", "True").replace("false", "False"))
# test_output_1

test_lyrics(test_lyric_1)

AttributeError: 'dict' object has no attribute 'replace'

In [17]:
%%time
test_lyric_2 = """
Breakup song, breakup song
Karde dil ki feeling strong
Saade chaar minute long
Breakup song, breakup song
Break break B B… Breakup song
Break break B B.. Breakup song
Angrezi chidiya ki khaatir
Desi dil mera tod diya
Maine chhod diya
Usey chhod diya
Uski kaali kartooton ne
Uska bhaanda phod diya
Maine chhod diya
Usey chhod diya
Dil pe patthar rakh ke
Munh pe makeup kar liya
Oh dil pe patthar rakh ke
Munh pe makeup kar liya
Mere saiyyan ji se aaj
Maine breakup kar liya
Mere saiyyan ji se aaj
Maine breakup kar liya
Subah savere uth ke maine
Ye sab kar liya
Mere saiyyan ji se aaj
Maine breakup kar liya
Humko bin bataye tune
Ye kab kar liya?
Arey humko bin bataye tune
Ye kab kar liya?
Oh tere saiyyan ji se kaahe
Tune break up kar liya
Tere saiyyan ji se kaahe
Tune breakup kar liya
Subah savere uth ke maine
Ye sab kar liya
Tere saiyyan ji se kaahe
Tune breakup kar liya
B b.. breakup song
Kuch din to rona dhona bumper kiya
Aur phir delete uska number kiya
Aansu jo sookhe seedha parlour gayi
Parlour mein jaake shampoo jamkar kiya
College ki saheliyon se catch-up kar liya
Arey college ki saheliyon se catch-up kar liya
Jinko mill na paayi unko whatsapp kar diya
Mere saiyyan ji se aaj maine breakup kar liya
Subah savere uth ke maine ye sab kar liya
Mere saiyyan ji se aaj maine breakup kar liya

Look, baby! mujhe lagta hai ki
Jo bhi tune kiya hai wo very very right hai
Bhoot-kaal ko bhool ja ab tu
Aane wala future very very bright hai
Main hoon na baby saath tere
Party sharty honi poori night hai
Mind na karna jo thoda zyada bol doon
Kyunki banda very very tight hai
Usey phone mila aur gaali de
Photo jala ke karde raakh
Saale teri maa ki aankh!
Kalti hua jo saiyan stupid tera
Jeevit hua hai phir se cupid tera
Baasi relationship ka label hata
Duniya ko tu hai available bata
Arey mere soye armaano ko wake up kar diya
Arey mere soye armaano ko wake up kar diya
Ke tere saiyan ji se (aaha!)
Tune breakup kar liya
Tere saiyyan ji se kaahe tune…
O tere saiyyan ji se kaahe tune
breakup kar liya
Tere saiyyan ji se kaahe tune
breakup kar liya
B b.. Breakup song…
Karde dil ki feeling strong…
"""

test_output_2 = query_llm(test_lyric_2)
test_output_2 = eval(test_output_2.replace("true", "True").replace("false", "False"))
test_output_2

CPU times: user 14min 24s, sys: 25.1 s, total: 14min 49s
Wall time: 1min 20s


{'Alert': False,
 'Excited': True,
 'Happy': False,
 'Content': True,
 'Relaxed': False,
 'Calm': False,
 'Bored': False,
 'Depressed': False,
 'Sad': False,
 'Distressed': False,
 'Angry': False,
 'Tense': False}

In [21]:
test_lyric_3 = """
bin pooche mera naam aur pata rasmon ko rakh ke pare chaar qadam bas chaar qadam chal do naa saath mere

bin pooche mera naam aur pata rasmon ko rakh ke pare chaar qadam bas chaar qadam chal do naa saath mere

bin kuch kahe bin kuch sune haathon mein haath liye chaar qadam bas chaar qadam chal do naa saath mere Hey bin kuch kahe bin kuch sune haathon mein haath liye chaar qadam bas chaar qadam chal do naa saath mere

raahoon mein tum ko jo dhoop sataae chhaanv bicha denge hum andhere daraayen to jaa kar falak pe chaand sajaa denge hum chhae udaasi latife sunaa kar tujh ko hansaa denge hum hanste-hansaate yoon hee gunagunaate chal denge chaar qadam

tum sa milee jo koyi rahaguzar duniyaa se kaun dare chaar qadam kya saaree umr chal duungi saath tere

bin kuch kahe bin kuch sune haathon mein haath liye chaar qadam bas chaar qadam chal do naa saath mere bin kuch kahe bin kuch sune haathon mein haath liye chaar qadam bas chaar qadam chal do naa saath mere
"""

test_output_3 = query_llm(test_lyric_3)
test_output_3 = eval(test_output_3.replace("true", "True").replace("false", "False"))
test_output_3

{'Alert': False,
 'Excited': True,
 'Happy': True,
 'Content': False,
 'Relaxed': False,
 'Calm': False,
 'Bored': False,
 'Depressed': False,
 'Sad': False,
 'Distressed': False,
 'Angry': False,
 'Tense': False}

In [23]:
test_lyric_4 = """
I took a pill in Ibiza
To show Avicii I was cool
And when I finally got sober, felt 10 years older
But fuck it, it was something to do
I'm living out in LA
I drive a sports car just to prove
I'm a real big baller cause I made a million dollars
And I spend it on girls and shoes

But you don't wanna be high like me
Never really knowing why like me
You don't ever wanna step off that roller coaster
And be all alone
You don't wanna ride the bus like this
Never knowing who to trust like this
You don't wanna be stuck up on that stage singing
Stuck up on that stage singing
All I know are sad songs, sad songs
Darling, all I know are sad songs, sad songs

I'm just a singer who already blew his shot
I get along with old timers
Cause my name's a reminder of a pop song people forgot
And I can't keep a girl, no
Cause as soon as the sun comes up
I cut 'em all loose and work's my excuse
But the truth is I can't open up

Now you don't wanna be high like me
Never really knowing why like me
You don't ever wanna step off that roller coaster
And be all alone
You don't wanna ride the bus like this
Never knowing who to trust like this
You don't wanna be stuck up on that stage singing
Stuck up on that stage singing
All I know are sad songs, sad songs
Darling, all I know are sad songs, sad songs

I took a plane to my home town
I brought my pride and my guitar
All my friends are all gone but there's manicured lawns
And the people still think I'm a star
I walked around downtown
I met some fans on Lafayette
They said tell us how to make it cause we're getting real impatient
So I looked 'em in the eye and said

You don't wanna be high like me
Never really knowing why like me
You don't ever wanna step off that roller coaster
And be all alone
You don't wanna ride the bus like this
Never knowing who to trust like this
You don't wanna be stuck up on that stage singing
Stuck up on that stage singing
All I know are sad songs, sad songs
Darling, all I know are sad songs, sad songs
"""

test_output_4 = query_llm(test_lyric_4)
test_output_4 = eval(test_output_4.replace("true", "True").replace("false", "False"))
test_output_4

{'Alert': False,
 'Excited': False,
 'Happy': False,
 'Content': True,
 'Relaxed': False,
 'Calm': False,
 'Bored': False,
 'Depressed': False,
 'Sad': True,
 'Distressed': False,
 'Angry': False,
 'Tense': False}

In [28]:
%%time
test_lyric_5 = """
Going out tonight, changes into something red Her mother doesn't like that kinda dress Everything she never had, she's showing off

Driving too fast, moon is breaking through her hair

She's heading for something that she won't forget Having no regrets is all that you really want We're only getting older, baby And I've been thinking about you lately

Does it ever drive you crazy Just how fast the night changes Everything you've ever dreamed of Disappearing when you wake up But there's nothing to be afraid of Even when the night changes It will never change me and you Chasing it tonight, doubts are running 'round her head He's waiting, hides behind his cigarette

Heart is beating loud, and she doesn't want it to stop Moving too fast, moon is lighting up her skin She's falling, doesn't even know it yet

Having no regrets is all that she really wants We're only getting older, baby And I've been thinking about you lately Does it ever drive you crazy


Having no regrets is all that she really wants We're only getting older, baby And I've been thinking about you lately Does it ever drive you crazy

Just how fast the night changes Everything that you've ever dreamed of Disappearing when you wake up But there's nothing to be afraid of Even when the night changes It will never change me and you Going out tonight, changes into something red Her mother doesn't like that kinda dress Reminds her of a missing piece of innocence she lost

We're only getting older baby And I've been thinking about you lately Does it ever drive you crazy Just how fast the night changes Everything that you've ever dreamed of Disappearing when you wake up But there's nothing to be afraid of Even when the night changes It will never change, baby It will never change, baby It will never change me and you
"""


test_output_5 = query_llm(test_lyric_5)
test_output_5 = eval(test_output_5.replace("true", "True").replace("false", "False"))
test_output_5

CPU times: user 7min 26s, sys: 12.7 s, total: 7min 39s
Wall time: 44.8 s


{'Alert': False,
 'Excited': True,
 'Happy': True,
 'Content': True,
 'Relaxed': False,
 'Calm': False,
 'Bored': False,
 'Depressed': False,
 'Sad': False,
 'Distressed': False,
 'Angry': False,
 'Tense': False}